In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import mord

In [2]:
data = pd.read_csv("Data_processed.csv")

In [3]:
data[:5]

,X,Date,Zone1Position,Zone2Position,Zone3Position,SKU,Zone1_Row_Num,Zone1_Col_Num,Zone2_Row_Num,Zone2_Col_num,...,Zone2_Humidity_Max,Zone2_Humidity_Range,Zone3_Humidity_Avg,Zone3_Humidity_Min,Zone3_Humidity_Max,Zone3_Humidity_Range,Block_Num,Block_Position,Block_Orientation,Result_Type
0,1,09/01/2020,3.0,2.0,3.0,A001,1.0,3.0,1.0,2.0,...,61.80,10.0,39.19,33.19,45.19,12.0,1000.0,1.0,1,Defect_1
1,2,03/12/2019,8.0,4.0,3.0,A001,2.0,4.0,2.0,2.0,...,58.12,16.0,48.50,40.50,56.50,16.0,1003.0,1.0,1,Defect_1
2,3,21/11/2019,4.0,2.0,3.0,B003,1.0,4.0,1.0,2.0,...,50.97,10.0,52.20,46.20,58.20,12.0,1001.0,4.0,1,PASS
3,4,22/11/2019,6.0,3.0,1.0,B003,2.0,2.0,2.0,1.0,...,59.13,4.0,39.29,31.29,47.29,16.0,1003.0,2.0,1,PASS
4,5,23/12/2019,3.0,2.0,3.0,B003,1.0,3.0,1.0,2.0,...,65.80,18.0,45.37,42.37,48.37,6.0,1002.0,1.0,1,PASS


In [4]:
data["Result_binary"] = np.where(data['Result_Type']== "PASS", 
                                 '0', '1')
data["Result_binary"].describe()

count     75750
unique        2
top           0
freq      54720
Name: Result_binary, dtype: object

In [5]:
data_model=data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", "Block_Position", 
                "Result_Type", "Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg","Zone3_Temp_Avg",
               "Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", "Zone3_Dur"]]

data_model2=data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", "Block_Position", 
                "Result_Type", "Zone3_Humidity_Range", "Zone2_Humidity_Range", "Zone1_Humidity_Range",
                  "Zone3_Temp_Range","Zone2_Temp_Range", "Zone1_Temp_Range", "Zone1_Dur", "Zone2_Dur", "Zone3_Dur"]]

#the choice of features for data_model2 is based on the results of the multinomial regression models (see the R script)

In [6]:
data_model.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75750 entries, 0 to 75749
Data columns (total 16 columns):
Zone1Position         75750 non-null float64
Zone2Position         75750 non-null float64
Zone3Position         75750 non-null float64
SKU                   75711 non-null object
Block_Num             75708 non-null float64
Block_Position        75711 non-null float64
Result_Type           75750 non-null object
Zone3_Humidity_Avg    75710 non-null float64
Zone2_Humidity_Avg    75715 non-null float64
Zone1_Humidity_Avg    75716 non-null float64
Zone3_Temp_Avg        75707 non-null float64
Zone2_Temp_Avg        75699 non-null float64
Zone1_Temp_Avg        75698 non-null float64
Zone1_Dur             75713 non-null float64
Zone2_Dur             75708 non-null float64
Zone3_Dur             75703 non-null float64
dtypes: float64(14), object(2)
memory usage: 9.2+ MB


In [7]:
data_model2.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75750 entries, 0 to 75749
Data columns (total 16 columns):
Zone1Position           75750 non-null float64
Zone2Position           75750 non-null float64
Zone3Position           75750 non-null float64
SKU                     75711 non-null object
Block_Num               75708 non-null float64
Block_Position          75711 non-null float64
Result_Type             75750 non-null object
Zone3_Humidity_Range    75750 non-null float64
Zone2_Humidity_Range    75750 non-null float64
Zone1_Humidity_Range    75750 non-null float64
Zone3_Temp_Range        75750 non-null float64
Zone2_Temp_Range        75750 non-null float64
Zone1_Temp_Range        75750 non-null float64
Zone1_Dur               75713 non-null float64
Zone2_Dur               75708 non-null float64
Zone3_Dur               75703 non-null float64
dtypes: float64(14), object(2)
memory usage: 9.2+ MB


In [8]:
train_data, test_data = train_test_split(data_model2.dropna(), test_size = 0.2)

1. Decision Tree

In [9]:
dt = tree.DecisionTreeClassifier()
x_train = pd.get_dummies(train_data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", 
                                     "Block_Position","Zone3_Humidity_Range", "Zone2_Humidity_Range", 
                                     "Zone1_Humidity_Range","Zone3_Temp_Range","Zone2_Temp_Range", "Zone1_Temp_Range",
                                     "Zone1_Dur", "Zone2_Dur","Zone3_Dur"]],drop_first=True)
y_train = train_data["Result_Type"]
dt = dt.fit(x_train, y_train)
x_test = pd.get_dummies(test_data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", 
                                     "Block_Position","Zone3_Humidity_Range", "Zone2_Humidity_Range", 
                                     "Zone1_Humidity_Range","Zone3_Temp_Range","Zone2_Temp_Range", "Zone1_Temp_Range",
                                     "Zone1_Dur", "Zone2_Dur","Zone3_Dur"]],drop_first=True)
y_test = test_data["Result_Type"]
y_pred = dt.predict(x_test)
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,Defect_1,Defect_2,Defect_3,Defect_4,Defect_5,PASS,All
True,,,,,,,
Defect_1,637,1,5,6,10,148,807
Defect_2,1,659,8,5,9,137,819
Defect_3,7,10,507,61,49,714,1348
Defect_4,4,5,44,59,30,459,601
Defect_5,8,5,35,39,47,478,612
PASS,166,135,775,688,644,8506,10914
All,823,815,1374,858,789,10442,15101


In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Defect_1       0.77      0.79      0.78       807
    Defect_2       0.81      0.80      0.81       819
    Defect_3       0.37      0.38      0.37      1348
    Defect_4       0.07      0.10      0.08       601
    Defect_5       0.06      0.08      0.07       612
        PASS       0.81      0.78      0.80     10914

    accuracy                           0.69     15101
   macro avg       0.48      0.49      0.48     15101
weighted avg       0.71      0.69      0.70     15101



<b> 2. Neural network </b> 

2. Logistic regression

In [51]:
data_model_logit=data[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", "Block_Position",
                       "Result_binary", "Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg",
                       "Zone3_Temp_Avg","Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", "Zone3_Dur"]]

In [ ]:
from sklearn.preprocessing import StandardScaler
train_data_lr, test_data_lr = train_test_split(data_model_logit, test_size = 0.2)
scaler = StandardScaler()
x_train_lr = scaler.fit_transform(pd.get_dummies(train_data_lr[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", 
                                     "Block_Position","Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg",
                                     "Zone3_Temp_Avg","Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", 
                                     "Zone3_Dur"]],drop_first=True))
                           
y_train_lr = train_data_lr["Result_binary"]
lr = LogisticRegression(random_state=0).fit(x_train_lr, y_train_lr)
x_test_lr = scaler.fit_transform(pd.get_dummies(test_data_lr[["Zone1Position", "Zone2Position", "Zone3Position", "SKU", "Block_Num", 
                                     "Block_Position","Zone3_Humidity_Avg", "Zone2_Humidity_Avg", "Zone1_Humidity_Avg",
                                     "Zone3_Temp_Avg","Zone2_Temp_Avg", "Zone1_Temp_Avg", "Zone1_Dur", "Zone2_Dur", 
                                     "Zone3_Dur"]],drop_first=True))
y_test_lr = test_data_lr["Result_binary"]
y_pred_lr = lr.predict(X_test_lr)
print(classification_report(y_test_lr, y_pred_lr))